In [1]:
import sys
import warnings
sys.path.append("../Pythonscripts")
sys.path.append(r"C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\Scripts")
import pandas as pd
import numpy as np
import synapseclient as sc
warnings.simplefilter("ignore")

In [2]:
## login to synapse
syn = sc.login()

Welcome, aryton tediarjo!



In [7]:
import sys
import warnings
import pandas as pd
import numpy as np
import synapseclient as sc
warnings.simplefilter("ignore")

DEMOGRAPHIC_TABLE_V1 = "syn8381056"
DEMOGRAPHIC_TABLE_V2 = "syn15673379"
WALKING_TABLE_V1     = "syn21021435"
WALKING_TABLE_V2     = "syn21018127"
BALANCE_TABLE_V1     = "syn21022189"
BALANCE_TABLE_V2     = "syn21018245"


def clean_data(version, demographic_table, table):
    if version == "V1":
        ## demographic data ## 
        demographic_entity = syn.get(demographic_table)
        demographic_data   = pd.read_csv(demographic_entity["path"], sep = "\t")[["healthCode", "PD"]]
    else:
        demographic_data   = syn.tableQuery("SELECT distinct(healthCode) as healthCode, \
                                             diagnosis as PD from {}".format(demographic_table))
        demographic_data   = demographic_data.asDataFrame().drop_duplicates("healthCode", keep = "first").reset_index(drop = True)
        demographic_data   = demographic_data[demographic_data["PD"] != "no_answer"] 
    demographic_data["PD"] = demographic_data["PD"].map({"parkinsons":1, "control":0})
    entity = syn.get(table)
    data   = pd.read_csv(entity["path"], index_col = 0)
    data = (data[data["phoneInfo"].str.contains("iPhone")]) \
                            [(data != "#ERROR").all(axis = 1)]
    data.drop_duplicates(subset=['healthCode', 'createdOn'], keep = "first", inplace = True)
    data[[_ for _ in data.columns if "feat" in _]] = \
    data[[_ for _ in data.columns if "feat" in _]].apply(pd.to_numeric)
    
    data.reset_index(drop = True, inplace = True)
    data = pd.merge(data, demographic_data, 
                    how = "inner", on = "healthCode")
    return data
    

def main():
    cleaned_PDKIT_MPV1 = clean_data("V1", DEMOGRAPHIC_TABLE_V1, WALKING_TABLE_V1)
    cleaned_PDKIT_MPV2 = clean_data("V2", DEMOGRAPHIC_TABLE_V2, WALKING_TABLE_V2)
    cleaned_SFM_MPV1 = clean_data("V1", DEMOGRAPHIC_TABLE_V2, BALANCE_TABLE_V1)
    cleaned_SFM_MPV2 = clean_data("V2", DEMOGRAPHIC_TABLE_V2, BALANCE_TABLE_V2)

In [4]:
entity = syn.get("syn21021435")
data   = pd.read_csv(entity["path"], index_col = 0)

In [10]:
test = "deviceMotion_walking_return.json.items__features_x.no_of_steps"

In [12]:
import re

<re.Match object; span=(40, 48), match='features'>

In [14]:
re.findall(r"\Bfeatures", test)

['features']

In [17]:
"userAccel{}".format(test.split("features")[1])

'userAccel_x.no_of_steps'

In [45]:
data_return   = data[[feature for feature in data.columns if "outbound" not in feature]]
data_outbound = data[[feature for feature in data.columns if "return" not in feature]]

In [48]:
strg = "deviceMotion_walking_return.json.items__features_x.no_of_steps"


def fix_column_name(data):
    
    
    for feature in filter(lambda x: "feature" in x, data.columns): 
        data  = data.rename({feature: "userAccel{}"\
                            .format(feature.split("features")[1])}, axis = 1)
    return data

data = 

In [50]:
pd.concat([fix_column_name(data_outbound), fix_column_name(data_return)])

,recordId,phoneInfo,createdOn,healthCode,userAccel_x.no_of_steps,userAccel_x.gait_step_regularity,userAccel_x.gait_stride_regularity,userAccel_x.gait_symmetry,userAccel_x.frequency_of_peaks,userAccel_x.max_freeze_index,...,userAccel_z.freeze_occurences,userAccel_z.speed_of_gait,userAccel_AA.no_of_steps,userAccel_AA.gait_step_regularity,userAccel_AA.gait_stride_regularity,userAccel_AA.gait_symmetry,userAccel_AA.frequency_of_peaks,userAccel_AA.max_freeze_index,userAccel_AA.freeze_occurences,userAccel_AA.speed_of_gait
0,674ab7db-c410-466b-9799-a1b9068224c8,iPhone 6 Plus,2015-03-09 19:50:29,1e0888df-7059-4fab-9dd7-b0b6616442e6,21.0,0.8105157863456157,0.9423873077245841,0.1318715213789684,122.112889933653,5.137270450592041,...,11.0,0.1657607191686766,10.0,0.9232905427869629,1.0,0.07670945721303712,569.1997437342815,3.0404622554779053,2.0,0.2079636602320212
1,7b556fcc-ebb8-46c3-b06a-0723243fc601,iPhone 6 Plus,2015-03-09 19:52:14,1e0888df-7059-4fab-9dd7-b0b6616442e6,19.0,0.820274401248235,0.9443241800229162,0.12404977877468115,69.90936317961213,2.736828327178955,...,0.0,0.21703207540961794,9.0,0.9057285028702519,1.0,0.09427149712974814,140.44907343867857,0.9738897681236267,0.0,0.2884299077199636
2,c32e3c41-1f5e-4b4f-9a5f-5f053ddd60ac,iPhone 6 Plus,2015-03-09 19:54:01,1e0888df-7059-4fab-9dd7-b0b6616442e6,19.0,0.7955362100423674,0.9375127981510041,0.14197658810863667,86.82865714514917,3.3860952854156494,...,17.0,0.17761305331803043,9.0,0.9306685271186917,1.0,0.06933147288130825,70.53539845189043,34.78596878051758,18.0,0.24424607115241348
3,ab490d44-c849-45d9-a841-8440a0d941a7,iPhone 6,2015-03-09 20:29:13,882b9ac4-ac7a-4108-9644-83d3544f28b4,11.0,0.7604081018677421,0.8185512936438587,0.05814319177611693,98.87489631293975,18.610525131225586,...,2.0,0.464821792681032,8.0,0.699939498636411,0.8361359609880834,0.13619646235163818,72.11691371119838,0.5801058411598206,0.0,0.3571769383724067
4,eb66b1f1-8f7a-4b51-8a0f-fbff899d5ca4,iPhone 6,2015-03-09 20:30:33,882b9ac4-ac7a-4108-9644-83d3544f28b4,15.0,0.7438869117342312,0.9194842793789971,0.17559736764476597,75.4924159947709,2.085009813308716,...,1.0,0.44460251870756673,7.0,0.5099771747565575,0.577540911923405,0.06756373716684794,233.12627794654748,0.7681419253349304,0.0,0.35041666884305717
5,cd78e98c-a9fa-4ff6-963f-74e2d6af8a17,iPhone 6,2015-03-09 20:48:07,416a984c-4a94-452b-93be-499c1ac374d6,4.0,1.0,1.0,0.0,725.045738232918,13.283235549926758,...,34.0,0.11156921224838791,6.0,1.0,1.0,0.0,34.80265108934724,1.7418649196624756,0.0,0.1612586222036049
6,a699e019-6772-401f-a837-1acb9bdb634e,iPhone 5s (GSM),2015-03-09 21:03:41,05c7ec40-2ea0-4714-910a-d818bd47e503,12.0,0.9470903238954336,1.0,0.05290967610456654,221.1096555739936,2.490642786026001,...,0.0,0.4127429222978438,11.0,0.4638000487074544,0.463800048707454,0.0,37.64547722119497,0.9303678870201111,0.0,0.26889340524767413
7,423b867c-dfaa-4e62-b7d5-40f474181fc4,iPhone 6,2015-03-09 21:04:20,3599ec25-58c9-4215-aac3-a1287f7a7601,9.0,0.9238821195495153,0.9238821195495154,0.0,81.44576997007277,2.0560317039489746,...,12.0,0.4317199392176604,5.0,0.7895904265568776,0.896964247455106,0.10737382089822833,56.34026506504063,1.8684746026992798,0.0,0.36204317118369095
8,1fe05bab-0c8f-48b5-8d20-412ecaccadcf,iPhone 5s (GSM),2015-03-09 21:05:24,05c7ec40-2ea0-4714-910a-d818bd47e503,13.0,0.9559760376890746,0.9559760376890746,0.0,131.36371982312278,2.452683687210083,...,0.0,0.36437116974454714,0.0,0.0,0.0,0.0,550.5792313093448,1.5415849685668945,0.0,0.24051174387269317
9,41095bb0-3cd8-439a-a3d2-d17a29865a84,iPhone 5s (GSM),2015-03-09 21:06:50,05c7ec40-2ea0-4714-910a-d818bd47e503,13.0,0.9567638898831411,0.9567638898831412,0.0,86.3406317376024,2.8371424674987793,...,0.0,0.3329751676653572,11.0,0.42654247808331414,0.42654247808331414,0.0,102.92371288365642,1.0319414138793945,0.0,0.2772329051647347
